In [1]:
from stable_baselines import DQN
from stable_baselines.gail import generate_expert_traj

model = DQN('MlpPolicy', 'CartPole-v1', verbose=1)
      # Train a DQN agent for 1e5 timesteps and generate 10 trajectories
      # data will be saved in a numpy archive named `expert_cartpole.npz`
generate_expert_traj(model, 'expert_cartpole', n_timesteps=int(1e5), n_episodes=10)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Creating environment from the given name, wrapped in a DummyVecEnv.






Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








--------------------------------------
| % time spent exploring  | 79       |
| episodes                | 100      |
| mean 100 episode reward | 21.3     |
| steps                   | 2111     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 17       |
| episodes     

{'actions': array([[0],
        [0],
        [0],
        ...,
        [1],
        [0],
        [1]]),
 'obs': array([[ 2.24449742e-03, -5.75635722e-03, -3.96197960e-02,
          1.58696324e-02],
        [ 2.12937035e-03, -2.00288370e-01, -3.93024050e-02,
          2.95793504e-01],
        [-1.87639717e-03, -3.94828618e-01, -3.33865359e-02,
          5.75826585e-01],
        ...,
        [ 2.33747888e+00,  1.32066762e+00, -7.42748603e-02,
         -6.31480440e-02],
        [ 2.36389208e+00,  1.51677155e+00, -7.55378231e-02,
         -3.78309667e-01],
        [ 2.39422750e+00,  1.32279909e+00, -8.31040144e-02,
         -1.10367715e-01]], dtype=float32),
 'rewards': array([1., 1., 1., ..., 1., 1., 1.]),
 'episode_returns': array([117., 111., 116., 112., 114., 110., 114., 117., 115., 110.]),
 'episode_starts': array([ True, False, False, ..., False, False, False])}

In [2]:
# pretraining using the saved trajectory

In [4]:
from stable_baselines import PPO2
from stable_baselines.gail import ExpertDataset
# Using only one expert trajectory
# you can specify `traj_limitation=-1` for using the whole dataset
dataset = ExpertDataset(expert_path='expert_cartpole.npz',
                        traj_limitation=-1, batch_size=128)

model = PPO2('MlpPolicy', 'CartPole-v1', verbose=1)
# Pretrain the PPO2 model
model.pretrain(dataset, n_epochs=1000)

# As an option, you can train the RL agent
# model.learn(int(1e5))

# Test the pre-trained model
env = model.get_env()
obs = env.reset()

reward_sum = 0.0
for _ in range(1000):
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        reward_sum += reward
        env.render()
        if done:
                print(reward_sum)
                reward_sum = 0.0
                obs = env.reset()

env.close()

actions (1136, 1)
obs (1136, 4)
rewards (1136,)
episode_returns (10,)
episode_starts (1136,)
Total trajectories: -1
Total transitions: 1136
Average returns: 113.6
Std for returns: 2.576819745345025
Creating environment from the given name, wrapped in a DummyVecEnv.
Pretraining with Behavior Cloning...
==== Training progress 10.00% ====
Epoch 100
Training loss: 0.515627, Validation loss: 0.569196

==== Training progress 20.00% ====
Epoch 200
Training loss: 0.467308, Validation loss: 0.514591

==== Training progress 30.00% ====
Epoch 300
Training loss: 0.404181, Validation loss: 0.448127

==== Training progress 40.00% ====
Epoch 400
Training loss: 0.333797, Validation loss: 0.398611

==== Training progress 50.00% ====
Epoch 500
Training loss: 0.278037, Validation loss: 0.357821

==== Training progress 60.00% ====
Epoch 600
Training loss: 0.248703, Validation loss: 0.311816

==== Training progress 70.00% ====
Epoch 700
Training loss: 0.236393, Validation loss: 0.294334

==== Training prog